---
# **LAB 6 - Audio effects**
---

In [ ]:
import os

# dirs
base_dir = './' 
wav_dir = os.path.join(base_dir, 'wav/') 
mp3_dir = os.path.join(base_dir, 'mp3/') 
out_dir = os.path.join(base_dir, 'output/') 

# print base dir 
print("Current dir:", os.getcwd())

# ▶️ Classes and functions

In [ ]:
import numpy as np

# local libraries
from src.pyaudio2 import *
from src.pyaudio3 import *
from src.pyaudio4 import *
from src.utils import *

def combFIR(gain, Fs, delay=1, delay_ms=None):
  """ First order delay FIR filter. 
      
      Params: 
        gain     : (float) gain (0 < a < 1) of the echo
        delay    : (int) delay (>0) in samples
        delay_ms : (float) delay (>0) in time
        name     : (strng) label
      
      Return:
        B, A : the filter coefficients
  """

  # delay in samples
  if delay_ms is None:
    D = delay - 1                    
  else:
    D = int(delay_ms * Fs) - 1  

  B = np.zeros(D+2)
  B[0], B[-1] = 1, gain
  A = [1]
  return B, A

def combIIR(gain, Fs, delay=1, delay_ms=None):
  """ First order delay IIR filter. 
      
      Params: 
        gain     : (float) gain (0 < a < 1) of the echo
        delay    : (int) delay in samples
        delay_ms : (float) delay in time
        name     : (strng) label
        
      Return:
        B, A : the filter coefficients
  """

  # delay in samples
  if delay_ms is None:
    D = delay - 1                    
  else:
    D = int(delay_ms * Fs) - 1  

  A = np.zeros(D+2)
  A[0], A[-1] = 1, -gain
  B = [1]
  return B, A

def allpass(gain, Fs, delay=1, delay_ms=None):
  """ Allpass filter. 
        
    Params: 
      gain     : (float) gain (0 < a < 1) 
      delay    : (int) delay in samples
      delay_ms : (float) delay in time
      name     : (strng) label

      
    Return:
      B, A : the filter coefficients
  """

  # delay in samples
  if delay_ms is None:
    D = delay - 1
  else:
    D = int(delay_ms * Fs) - 1 
  
  B = np.append(np.append(gain, np.zeros(D)), 1)
  A = np.append(np.append(1, np.zeros(D)), gain)
  return B, A

def schroeder(wave, cg, cd, ag, ad, Fs):
  """ Reverberator based on Schroeder's design which consists of 4 parallel 
      feedback comb filters in series with 2 cascaded all pass filters.
      
      Params: 
        wave     : (Digital) the input signal
        cg       : (list) 4 comb filter gains (less than 1 for stability)	
        cd       : (list) 4 comb filter delays 
        ag       : (list) 2 allpass filter gains (less than 1 for stability)
        ad       : (list) 2 allpass filter delays 
        gain     : (scalar) the gain factor of the direct signal
      
      Return:
        A (Digital) reverb signal
  """
  
  # comb design
  B0,A0 = combIIR(cg[0], Fs, delay=cd[0])
  B1,A1 = combIIR(cg[1], Fs, delay=cd[1])
  B2,A2 = combIIR(cg[2], Fs, delay=cd[2])
  B3,A3 = combIIR(cg[2], Fs, delay=cd[3])
  
  # comb filters
  comb0 = Filter(B0, A0, Fs) 
  comb1 = Filter(B1, A1, Fs) 
  comb2 = Filter(B2, A2, Fs) 
  comb3 = Filter(B3, A3, Fs) 

  # apply filters
  w0 = comb0.apply(wave)   # filtered signal
  w1 = comb1.apply(wave)   # filtered signal
  w2 = comb2.apply(wave)   # filtered signal
  w3 = comb3.apply(wave)   # filtered signal

  # sum all signals
  wave_comb = w0 + w1 + w2 + w3
  wave_comb.normalize()

  # allpass design 
  B0,A0 = allpass(ag[0], Fs, delay=ad[0])
  B1,A1 = allpass(ag[1], Fs, delay=ad[1])

  # comb filters
  allp0 = Filter(B0, A0, Fs) 
  allp1 = Filter(B1, A1, Fs) 

  # apply filters
  wave_allp_comb = allp0.apply(wave_comb)        # filtered signal
  wave_allp_comb = allp1.apply(wave_allp_comb)   # filtered signal
  wave_allp_comb.normalize()

  return wave_allp_comb

def flanger(x, Fs, max_delay=0.01, Fd=1):
  """ Flanger time-varying filter
    
    Params: 
      x         : (array) samples
      Fs        : (int) sample rate
      max_delay : (float) max delay in seconds 
      Fd        : (float) rate of flange in Hz
    
    Return:
      (Digital) waveform of the delayed (flanged) samples
  """

  N = len(x)   # tot samples

  # convert delay in ms to max delay in samples
  D = round(max_delay*Fs)

  # low frequency oscillating delay (cast to int)
  n = np.linspace(0,N,N)
  LFO = 1 - np.cos(2 * np.pi * n * (Fd/Fs))
  d = (D/2*LFO).astype(int) # delays

  # create empty out vector
  y = np.zeros(N)      

  # to avoid referencing of negative samples
  y[1:D] = x[1:D] 

  amp = 0.7    # suggested coefficient from page 71 DAFX

  # for each sample
  for i in range(D+1,N):
    y[i] = amp * (x[i] + x[i-d[i]])  # add delayed sample
  
  # output
  wave_fla = Digital(y, Fs) # create a Digital waveform
  wave_fla.normalize()
  return wave_fla


# ✅ Comb filters 

In [ ]:
# delay FIR filter
G = 0.9   # gain of the echo
D = 10     # samples
Fs = 100  # Hz
B, A = combFIR(G, Fs, delay=D)
comb = Filter(B, A, Fs, name='Comb FIR') 
print(comb)

# analysis
comb.plot_mag_phase()
comb.plot_pole_zero()
comb.plot_impulse_resp()

Using comb FIR filter to delay speech

In [ ]:
# acoustic music
acoustic_file = wav_dir + 'speech3.wav'   # 'acoustic.wav'

# audio player
ap = Audio()
ap.read_wav(acoustic_file)
ap.play()

# delay FIR filter
G = 0.9       # gain of the echo
D = 22050     # .5 sec delay in samples
Fs = ap.samplerate   
B, A = combFIR(G, Fs, delay=D)
comb = Filter(B, A, Fs, name='Comb FIR') 

# get sound and filter it
music = ap.get_Digital()         # get Digital object from Audio object
music_fil = comb.apply(music)   # filtered signal: apply to annoy sound
ap1 = Audio(music_fil)
ap1.play()

## 🔴 TODO

* Design a IIR comb filter with the same parameters
* Apply the filter to the previous speech and compare the results with the comb FIR filter

In [ ]:
### TODO

# delay IIR filter

# analysis

# design for the speech

# get sound and filter it

In [ ]:
### SOLUTION

# delay IIR filter

G = 0.6   # gain of the echo
D = 10    # samples
Fs = 100  # Hz
B, A = combIIR(G, Fs, delay=D)
comb = Filter(B, A, Fs, name='Comb IIR') 
print(comb)

# analysis
comb.plot_mag_phase()
comb.plot_pole_zero()
comb.plot_impulse_resp()

# design for the speech
G = 0.9   # gain of the echo
D = 22050     # samples
Fs = ap.samplerate   
B, A = combIIR(G, Fs, delay=D)
comb = Filter(B, A, Fs, name='Comb IIR') 
comb.plot_mag_phase()

# get sound and filter it
music = ap.get_Digital()         # get Digital object from Audio object
music_fil = comb.apply(music)   # filtered signal: apply to annoy sound
ap1 = Audio(music_fil)
ap1.play()

# ✅ Allpass filter 

In [ ]:
# delay FIR filter
G = 0.7   # gain of the echo
D = 10    # samples
Fs = 100  # Hz
  
B, A = allpass(G, Fs, delay=D)
all = Filter(B, A, Fs, name='Allpass') 
print(all)

# analysis
all.plot_mag_phase(npoints=10000)
all.plot_pole_zero()
all.plot_impulse_resp()


In [ ]:
# acoustic music
acoustic_file = wav_dir + 'speech3.wav'   # 'acoustic.wav'

# audio player
ap = Audio()
ap.read_wav(acoustic_file)
ap.play()

# allpass filter
G = 0.7      # gain of the echo
D = 800      # ~.05 sec in samples
Fs = ap.samplerate   
B, A = allpass(G, Fs, delay=D)
all = Filter(B, A, Fs, name='Allpass') 
print(all)

# get sound and filter it
music = ap.get_Digital()       # get Digital object from Audio object
music_fil = all.apply(music)   # filtered signal: apply to annoy sound
ap1 = Audio(music_fil)
ap1.play()

# ✅ Shroeder reverb

Implement a Schroeder's filter with parameters:
```
cg = [.9, .8, .7, .75]
cd = [100, 200, 300, 400]
ag = [.9, .8]
ad = [100, 200]
```

In [ ]:
# acoustic music
acoustic_file = wav_dir + 'speech3.wav' 

# audio player
ap = Audio()
ap.read_wav(acoustic_file)
ap.play()

# get wave and apply reverb filter
cg = [.9, .8, .7, .75]
cd = [100, 200, 300, 400]
ag = [.9, .8]
ad = [100, 200]
wave = ap.get_Digital()
Fs = ap.samplerate
wave_rev = schroeder(wave, cg, cd, ag, ad, Fs)

# play filtered
a = Audio(wave_rev)
a.play()

## 🔴 TODO

Apply the same filter to the `acoustic.wav` sound with parameters:
```
cg = [.7, .7, .7, .7]
cd = [16870, 16010, 30530, 35530]
ag = [.7, .7]
ad = [347, 113]
```

In [ ]:
#### TODO

# acoustic music
acoustic_file = wav_dir + 'acoustic.wav'

# audio player

# get wave and apply reverb filter

# play filtered

In [ ]:
#### SOLUTION

# acoustic music
acoustic_file = wav_dir + 'violin.wav'

# audio player
ap = Audio()
ap.read_wav(acoustic_file)
ap.play()

# get wave and apply reverb filter
cg = [.7, .7, .7, .7]
cd = [16870, 16010, 30530, 35530]
ag = [.7, .7]
ad = [347, 113]
wave = ap.get_Digital()
Fs = ap.samplerate
wave_rev = schroeder(wave, cg, cd, ag, ad, Fs)

# play filtered
a = Audio(wave_rev)
a.play()

# ✅ Flanger




Creates a single delay with the delay time oscillating within $0-10$ ms at $0.1 - 5$ Hz.

this is not synthesisable unless buffering is used do calculations with sampling frequency to convert delay in samples into miliseconds

In [ ]:
# waveform of a note played by the Piano
acoustic_file = wav_dir + 'acoustic.wav'

# play original
a = Audio()
a.read_wav(acoustic_file)
a.play()

# Flanger filter
wave_fla = flanger(a.audio, a.samplerate, max_delay=0.01, Fd=1)

# play filtered
a1 = Audio(wave_fla)
a1.play()

# plot all signals
multiplot([a.audio, a1.audio])

# ✅ Chorus

Chorus is an effect that simulates the presence of several sources playing in imperfect unison. It is implemented by combining the original signal with several copies delayed by a randomly varied number of samples. Typical delays are in the range D = 10 - 25 ms, and they are modified by small, slowly varying random variations.


def chorus2(x, Fs, Fc=0.01, amp=0.2):
  """ Chorus filter effect for 2 sounds.
    
    Params: 
      x    : (array) samples
      Fs   : (int) sample rate
      Fc   : (float)  rate of variation in [cycle/samp]
      amp  : (float) suggested coefficient from page 71 DAFX
    
    Return:
      (Digital) waveform of the delayed (flanged) samples
  """

# 🔴 TODO

In [ ]:
#### TODO
def chorus2(x, Fs, Fc=0.01, amp=0.2):
  """ Chorus filter effect for 2 sounds.
    
    Params: 
      x    : (array) samples
      Fs   : (int) sample rate
      Fc   : (float)  rate of variation in [cycle/samp]
      amp  : (float) suggested coefficient from page 71 DAFX
    
    Return:
      (Digital) waveform of the delayed (flanged) samples
  """


In [ ]:
#### TODO

# waveform of a note played by the Piano
acoustic_file = wav_dir + 'acoustic.wav'

# play original

# Chorus filter

# play filtered

# plot all signals


In [ ]:
#### SOLUTION

def chorus2(x, Fs, Fc=0.01, amp=0.2):
  """ Chorus filter effect for 2 sounds.
    
    Params: 
      x    : (array) samples
      Fs   : (int) sample rate
      Fc   : (float)  rate of variation in [cycle/samp]
      amp  : (float) suggested coefficient from page 71 DAFX
    
    Return:
      (Digital) waveform of the delayed (flanged) samples
  """

  N = len(x)   # tot samples

  D = np.round(1/Fc).astype(int)   # period of random generator
  N = int(N - np.mod(N, D))        # tot samples (multip. of D)
  x = x[1:N+1]                       # cut samples 

  # random variations for the 2 sounds
  R = int(N * Fc)
  
  r1 = np.random.rand(1,R)-0.5   # zero-mean random numbers
  r2 = np.random.rand(1,R)-0.5   # zero-mean random numbers
  r1 = r1.squeeze()
  r2 = r2.squeeze()

  # create 2 empty delays vectors
  d1 = np.zeros(N).astype(int)
  d2 = np.zeros(N).astype(int)

  # random delays using linear interpolation in between
  for i in range(1,R-1):
    # sound first delays
    line = r1[i]+(r1[i+1]-r1[i]) * np.arange(0,D)/D
    d1[(i-1)*D:i*D] = np.round(D*(0.5 + line)).astype(int)
    
    # sound second delays
    line = r2[i]+(r2[i+1]-r2[i])*np.arange(0,D)/D
    d2[(i-1)*D:i*D] = np.round(D*(0.5 + line)).astype(int)

  # create empty out vector
  y = np.zeros(N)      

  # to avoid referencing of negative samples
  y[1:D] = x[1:D]

  # for each sample
  for i in range(D+1, N):

    # first random delayed sound
    y1 = x[i-d1[i]]
    
    # second random delayed sound
    y2 = x[i-d2[i]]
    
    # sum up
    y[i] = x[i] + amp*(y1+y2)   # add delayed sample

  # output
  wave_cho = Digital(y, Fs) # create a Digital waveform
  wave_cho.normalize()
  return wave_cho

In [ ]:
# waveform of a note played by the Piano
acoustic_file = wav_dir + 'acoustic.wav'

# play original
ap = Audio()
ap.read_wav(acoustic_file)
ap.play()

# Chorus filter
wave_cho = chorus2(ap.audio, a.samplerate, Fc=0.02, amp=0.2)

# play filtered
a = Audio(wave_cho)
a.play()

# plot all signals
multiplot([ap.audio, a.audio])